In [50]:
from pyspark.sql import SparkSession 

In [51]:
spark = SparkSession .\
builder .\
appName('spark') .\
config("spark.sql.waerhouse.dir","/user/itv016843/warehouse") .\
enableHiveSupport() .\
master("yarn") .\
getOrCreate()

In [52]:
spark

In [55]:
spark.sql("use data_analyst")

""


In [56]:
spark.sql("show tables")

database,tableName,isTemporary
data_analyst,customers,false
data_analyst,loans,false
data_analyst,loans_defaulters_...,false
data_analyst,loans_defaulters_...,false
data_analyst,loans_repayments,false
data_analyst,qa_data,false


In [57]:
df2 = spark.sql("select member_id,count(*) as total from customers group by member_id having count(*) > 1 order by total desc")

In [58]:
df2.write \
.format("csv") \
.mode("overwrite") \
.option("header",True) \
.option("path","/user/itv016843/bad1") \
.save()

In [59]:
df3 = spark.sql("select member_id,count(*) as total from loans_defaulters_delinq group by member_id having count(*) > 1 order by total desc")

In [60]:
df3.count()

173

In [61]:
df3.write \
.format("csv") \
.mode("overwrite") \
.option("header",True) \
.option("path","/user/itv016843/bad2") \
.save()

In [62]:
df4 = spark.sql("select member_id,count(*) as total from loans_defaulters_records_enq group by member_id having count(*) > 1 order by total desc")

In [64]:
df4.write \
.format("csv") \
.mode("overwrite") \
.option("header",True) \
.option("path","/user/itv016843/bad3") \
.save()

In [65]:
df5 = spark.read \
.format("csv") \
.option("header",True) \
.load("/user/itv016843/bad1")


In [66]:
df5.createOrReplaceTempView("bad1")

In [67]:
df6 = spark.read \
.format("csv") \
.option("header",True) \
.load("/user/itv016843/bad2")


In [68]:
df6.createOrReplaceTempView("bad2")

In [69]:
df7 = spark.read \
.format("csv") \
.option("header",True) \
.load("/user/itv016843/bad3")

In [70]:
df7.createOrReplaceTempView("bad3")

In [71]:
spark.sql("show tables")

database,tableName,isTemporary
data_analyst,customers,false
data_analyst,loans,false
data_analyst,loans_defaulters_...,false
data_analyst,loans_defaulters_...,false
data_analyst,loans_repayments,false
data_analyst,qa_data,false
,bad1,true
,bad2,true
,bad3,true


In [72]:
df5 = df2.select("member_id") \
.union(df3.select("member_id")) \
.union(df4.select("member_id"))

In [76]:
df6 = df5.distinct()

In [81]:
df6.write \
.format("csv") \
.mode("overwrite") \
.option("header",True) \
.option("path","/user/itv016843/bad_data") \
.save()

In [80]:
spark.sql("show tables")

database,tableName,isTemporary
data_analyst,customers,false
data_analyst,loans,false
data_analyst,loans_defaulters_...,false
data_analyst,loans_defaulters_...,false
data_analyst,loans_repayments,false
data_analyst,qa_data,false
,bad1,true
,bad2,true
,bad3,true


In [82]:
bad = spark.read \
.format("csv") \
.option("header",True) \
.load("/user/itv016843/bad_data")

In [90]:
bad.limit(5)

member_id
11099eff1a2ced61f...
b0b069b23773292a8...
2e6ad5a0aaaf31adb...
793681798e3c70fb0...
8347b69d3676e357a...


In [85]:
bad.createOrReplaceTempView("badd")

In [86]:
spark.sql("show tables")

database,tableName,isTemporary
data_analyst,customers,false
data_analyst,loans,false
data_analyst,loans_defaulters_...,false
data_analyst,loans_defaulters_...,false
data_analyst,loans_repayments,false
data_analyst,qa_data,false
,bad1,true
,bad2,true
,bad3,true
,badd,true


In [89]:
final = spark.sql("select * from customers where member_id not in(select member_id from badd)")

In [91]:
final.filter("member_id like '11099eff1a2ced61f%' ")

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
